In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import seaborn as sns
import warnings
import sklearn
from sklearn import impute

In [3]:
train_transaction = pd.read_csv('/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/train_identity.csv")
test_transaction = pd.read_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/test_transaction.csv")
test_identity = pd.read_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/test_identity.csv")

train_data = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
test_data = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')


train_fraud = train_data[train_data.isFraud == 1]
train_nofraud = train_data[train_data.isFraud == 0]

In [4]:
test_data.columns = test_data.columns.str.replace('-','_')

## Memory usage reduction

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2 # just added 
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

In [6]:
reduce_mem_usage(train_data)

Mem. usage decreased from 2598.36 Mb to 1288.96 Mb (50.4% reduction)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.500000,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.000000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.000000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.000000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.000000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500000,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.953125,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Delete columns with too much missing data or repetition

In [7]:
def get_too_many_null_attr(data):
    many_null_cols = [col for col in data.columns if data[col].isnull().sum() / data.shape[0] > 0.9]
    return many_null_cols

In [8]:
def get_too_many_repeated_val(data):
    big_top_value_cols = [col for col in train_data.columns if train_data[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
    return big_top_value_cols

In [9]:
def get_useless_columns(data):
    too_many_null = get_too_many_null_attr(data)
    print("More than 90% null: " + str(len(too_many_null)))
    too_many_repeated = get_too_many_repeated_val(data)
    print("More than 90% repeated value: " + str(len(too_many_repeated)))
    cols_to_drop = list(set(too_many_null + too_many_repeated))
    cols_to_drop.remove('isFraud')
    return cols_to_drop

In [10]:
cols_to_drop = get_useless_columns(train_data)

More than 90% null: 12
More than 90% repeated value: 67


In [11]:
cols_to_drop = get_useless_columns(train_data)

More than 90% null: 12
More than 90% repeated value: 67


In [12]:
train_data = train_data.drop(cols_to_drop, axis = 1)

In [13]:
test_data = test_data.drop(cols_to_drop, axis = 1)

## Separate categorical and numerical features

In [14]:
num_cols = list(train_data.select_dtypes(include = ['int64', 'int8', 'int16', 'int32', 'float32', 'float16', 'float64']).columns)

num_cols_test = num_cols

num_cols_test.remove('isFraud')

num_cols_test

['TransactionID',
 'TransactionDT',
 'TransactionAmt',
 'card1',
 'card2',
 'card3',
 'card5',
 'addr1',
 'addr2',
 'dist1',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V8

In [15]:
cat_cols = list(train_data.select_dtypes(include=['object']).columns)

cat_cols

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_16',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

## We replace infinite values with 999

In [16]:
train_data = train_data.replace(np.inf,999)
test_data = test_data.replace(np.inf,999)

## We impute missing values with median (in case of numerical features) or mode (in case of categorical features)

In [17]:
from sklearn.impute import SimpleImputer, KNNImputer


imp_median = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')



### Train data / NaN imputation


In [18]:
imp_mode.fit(train_data[cat_cols])

SimpleImputer(strategy='most_frequent')

In [19]:
imp_median.fit(train_data[num_cols])

SimpleImputer(strategy='median')

In [20]:
train_data[cat_cols] = imp_mode.transform(train_data[cat_cols])



In [21]:
train_data[num_cols] = imp_median.transform(train_data[num_cols])

### Test data / NaN imputation


In [22]:
imp_mode.fit(test_data[cat_cols])

SimpleImputer(strategy='most_frequent')

In [23]:
imp_median.fit(test_data[num_cols_test])

SimpleImputer(strategy='median')

In [24]:
test_data[cat_cols] = imp_mode.transform(test_data[cat_cols])

In [25]:
test_data[num_cols_test] = imp_median.transform(test_data[num_cols_test])

In [26]:
summary = pd.DataFrame(train_data[num_cols].dtypes, columns = ['Data Type'])
summary = summary.reset_index()
summary['Name'] = summary['index']
summary = summary.drop(columns=['index'])
summary['Missing(%)'] = train_data[num_cols].isnull().sum().values / train_data[num_cols].shape[0] * 100
summary['unique values'] = train_data[num_cols].nunique().values
summary


,Data Type,Name,Missing(%),unique values
0,float64,TransactionID,0.0,590540
1,float64,TransactionDT,0.0,573349
2,float64,TransactionAmt,0.0,8195
3,float64,card1,0.0,13553
4,float64,card2,0.0,500
...,...,...,...,...
333,float64,id_14,0.0,25
334,float64,id_17,0.0,104
335,float64,id_19,0.0,522
336,float64,id_20,0.0,394


In [27]:
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000.0,0,86400.0,68.5,W,13926.0,361.0,150.0,discover,142.0,...,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,Windows
1,2987001.0,0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,...,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,Windows
2,2987002.0,0,86469.0,59.0,W,4663.0,490.0,150.0,visa,166.0,...,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,Windows
3,2987003.0,0,86499.0,50.0,W,18132.0,567.0,150.0,mastercard,117.0,...,chrome 63.0,24.0,1920x1080,match_status:2,T,F,T,F,desktop,Windows
4,2987004.0,0,86506.0,50.0,H,4497.0,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


## Encoding train data


In [28]:
from sklearn import preprocessing
import category_encoders

X_le = train_data.copy()
for f in train_data.columns:
    if X_le[f].dtype == 'object': 
        le = preprocessing.LabelEncoder()
        le.fit(list(X_le[f].values))
        X_le[f] = le.transform(list(X_le[f].values))

X_be = train_data.copy()
for f in train_data.columns:
    if X_be[f].dtype == 'object': 
        if X_be[f].nunique() <= 2:
            le = preprocessing.LabelEncoder()
            le.fit(list(X_be[f].values))
            X_be[f] = le.transform(list(X_be[f].values))
        else:
            be = category_encoders.BinaryEncoder(cols=f)
            X_be = be.fit_transform(X_be)

## Encoding test data

In [29]:
X_le_test = test_data.copy()
for f in test_data.columns:
    if X_le_test[f].dtype == 'object': 
        le = preprocessing.LabelEncoder()
        le.fit(list(X_le_test[f].values))
        X_le_test[f] = le.transform(list(X_le_test[f].values))

X_be_test = test_data.copy()
for f in test_data.columns:
    if X_be_test[f].dtype == 'object': 
        if X_be_test[f].nunique() <= 2:
            le = preprocessing.LabelEncoder()
            le.fit(list(X_be_test[f].values))
            X_be_test[f] = le.transform(list(X_be_test[f].values))
        else:
            be = category_encoders.BinaryEncoder(cols=f)
            X_be_test = be.fit_transform(X_be_test)

In [30]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


In [31]:
X_le.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000.0,0,86400.0,68.5,4,13926.0,361.0,150.0,1,142.0,...,47,24.0,132,3,1,0,1,0,0,1598
1,2987001.0,0,86401.0,29.0,4,2755.0,404.0,150.0,2,102.0,...,47,24.0,132,3,1,0,1,0,0,1598
2,2987002.0,0,86469.0,59.0,4,4663.0,490.0,150.0,3,166.0,...,47,24.0,132,3,1,0,1,0,0,1598
3,2987003.0,0,86499.0,50.0,4,18132.0,567.0,150.0,2,117.0,...,47,24.0,132,3,1,0,1,0,0,1598
4,2987004.0,0,86506.0,50.0,1,4497.0,514.0,150.0,2,102.0,...,123,32.0,164,3,1,0,1,1,1,954


In [32]:
X_le_test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549.0,18403224.0,31.95,4,10409.0,111.0,150.0,3,226.0,2,...,53,24.0,171,1,1,0,1,0,0,2029
1,3663550.0,18403263.0,49.00,4,4272.0,111.0,150.0,3,226.0,2,...,53,24.0,171,1,1,0,1,0,0,2029
2,3663551.0,18403310.0,171.00,4,4476.0,574.0,150.0,3,226.0,2,...,53,24.0,171,1,1,0,1,0,0,2029
3,3663552.0,18403310.0,284.95,4,10989.0,360.0,150.0,3,166.0,2,...,53,24.0,171,1,1,0,1,0,0,2029
4,3663553.0,18403317.0,67.95,4,18018.0,452.0,150.0,2,117.0,2,...,53,24.0,171,1,1,0,1,0,0,2029


In [33]:
x_le = X_le.drop('isFraud', axis = 1 )
y_le = X_le['isFraud']

x_be = X_be.drop('isFraud', axis = 1)
y_be = X_be['isFraud']

In [34]:
x_le.shape

(590540, 367)

## Loading the test data set and creating submisssion dataframe

In [35]:
test_transaction = pd.read_csv('/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/test_transaction.csv')
submission = pd.DataFrame(test_transaction["TransactionID"])
submission_RF = pd.DataFrame(test_transaction["TransactionID"])
submission_be = pd.DataFrame(test_transaction["TransactionID"])
submission_RF_be = pd.DataFrame(test_transaction["TransactionID"])
del test_transaction

## Creating Y_train, X_train

In [ ]:
pd.set_option('display.max_rows', 400)

In [37]:
Y_train = train_data["isFraud"]
X_train = train_data.drop("isFraud", axis = 1)
Xle_train = X_le.drop("isFraud", axis = 1)
Xbe_train = X_be.drop("isFraud", axis = 1)

In [37]:
X_train.dtypes

TransactionID     float64
TransactionDT     float64
TransactionAmt    float64
ProductCD          object
card1             float64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
dist1             float64
P_emaildomain      object
R_emaildomain      object
C1                float64
C2                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               float64
C14               float64
D1                float64
D2                float64
D3                float64
D4                float64
D5                float64
D6                float64
D8                float64
D9                float64
D10               float64
D11               float64
D12         

In [38]:
train_data.dtypes

TransactionID     float64
isFraud              int8
TransactionDT     float64
TransactionAmt    float64
ProductCD          object
card1             float64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
dist1             float64
P_emaildomain      object
R_emaildomain      object
C1                float64
C2                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               float64
C14               float64
D1                float64
D2                float64
D3                float64
D4                float64
D5                float64
D6                float64
D8                float64
D9                float64
D10               float64
D11         

## PCA

In [90]:
from sklearn.decomposition import PCA
pca = PCA()
x_pca = pd.DataFrame(pca.fit_transform(x_le))
x_pca.head(5)

,0,1,2,3,4,5,6,7,8,9,...,423,424,425,426,427,428,429,430,431,432
0,-7.291825e+06,-4784.363056,-19552.738405,2112.186754,-27246.536504,-4004.573547,206.073073,13.572158,-250.265983,-735.641398,...,0.000198,-0.000120,0.000019,0.000121,-0.000030,0.000060,-0.000158,-0.000004,0.000073,-2.042707e-05
1,-7.291824e+06,-4782.518486,-19556.412829,1772.174087,-27223.253503,7162.116210,172.333816,-39.229217,-195.920422,-717.939808,...,0.000043,-0.000150,0.000018,0.000118,0.000131,-0.000107,0.000025,-0.000036,0.000038,-1.464809e-05
2,-7.291756e+06,-4782.722398,-19555.711392,1808.206588,-27229.816703,5254.593820,166.012685,-46.152792,-205.222031,-719.583724,...,0.000142,-0.000094,0.000055,0.000273,0.000150,-0.000070,0.000051,-0.000078,0.000067,-2.804844e-05
3,-7.291726e+06,-4817.523297,-19552.848894,4510.555360,-27132.598806,-8176.913738,1499.589923,1475.122888,42.374934,-857.297017,...,-0.001125,-0.000681,0.000628,0.000119,0.000189,0.000003,-0.000013,-0.000655,-0.000809,-2.307865e-05
4,-7.292075e+06,-71846.484613,146341.408423,741.795584,-21874.557966,5439.527972,153.460947,-113.103650,99.366687,6220.953007,...,-0.000483,0.001004,0.000238,-0.000139,-0.000169,0.000156,0.000615,0.000257,-0.000662,-7.635384e-07


In [44]:
x_test_pca = pd.DataFrame(pca.fit_transform(X_le_test.iloc[0:1000]))

In [ ]:
my_var = pca.explained_variance_ratio_
my_var
sum(my_var[0:10])

In [91]:
pca_xtrain = x_pca.iloc[:,0:10]
pca_ytrain = y_le
pca_test = x_test_pca.iloc[:,10]

## Logistic regression

In [43]:
lr = LogisticRegression(penalty='l2', max_iter=500, n_jobs=6, tol=1e-6, solver="sag")
lr.fit(x_le, np.ravel(y_le))


/Users/szeberinricsi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(max_iter=500, n_jobs=6, solver='sag', tol=1e-06)

In [44]:
Yhat_lr = lr.predict_proba(X_le_test)
submission["isFraud"] = Yhat_lr[:, 1]


In [45]:
submission.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionLR.csv", index = False)

## Random forest

### Parameters

In [ ]:
nFeatures = x_le.shape[1]
oobErrList = list()
mList = [m for m in range(10, nFeatures+1, 30)]

for m in mList:
    rf = RandomForestClassifier(max_features=m, min_samples_leaf=1,\
                                oob_score=True, n_estimators=50)
    rf.fit(x_le, np.ravel(y_le))
    oobErrList.append(1-rf.oob_score_)
    print(m, 1-rf.oob_score_)

10 0.02025773021302535


In [40]:
rfauto = RandomForestClassifier(max_features="auto", min_samples_leaf=1, n_estimators=1000)
rfauto.fit(x_le, np.ravel(y_le))
Yhat_rfauto = rfauto.predict_proba(X_le_test)




NameError: name 'Yhat_rf' is not defined

In [ ]:
treeCount = 1000
m = 15

rf = RandomForestClassifier(max_features=m, min_samples_leaf=1, n_estimators=treeCount)
rf.fit(x_le, np.ravel(y_le))


In [1]:
Yhat_rf = rf.predict_proba(X_le_test)

NameError: name 'rf' is not defined

In [41]:
submission_RF["isFraud"] = Yhat_rfauto[:, 1]
submission_RF.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionRFauto.csv", index = False)


In [ ]:
submission_RF["isFraud"] = Yhat_rf[:, 1]

In [ ]:
submission

In [ ]:
submission_RF.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionRF.csv", index = False)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#instanciation
knn = KNeighborsClassifier(n_neighbors=600)
#training
#for i in range (300, 1200, 300):
knn.fit(x_le.iloc[0:3000], y_le.iloc[0:3000])

#prediction 
prediction = pd.DataFrame(knn.predict(X_le_test))


In [ ]:
prediction.describe()

## models with pCA

In [ ]:
lr = LogisticRegression(penalty='l2', max_iter=500, n_jobs=6, tol=1e-6, solver="sag")
lr.fit(pca_xtrain, np.ravel(y_le))

In [93]:
treeCount = 1000
m = 15

rf = RandomForestClassifier(max_features=10, min_samples_leaf=1, n_estimators=treeCount)
rf.fit(pca_xtrain, np.ravel(pca_ytrain))

Yhat_pca_rf = lr.predict_proba(pca_test)
submission["isFraud"] = Yhat_pca_rf[:, 1]

ValueError: Expected 2D array, got 1D array instead:
array=[ 3.97793526e+02  4.15590066e+02  5.00897341e+01  7.11441978e+01
 -4.08246984e+02 -2.97935420e+02 -4.48034698e+02 -2.05121932e+02
  5.12666460e+02 -1.85680726e+02 -2.46278909e+02 -3.59339924e+01
 -3.02513968e+02 -1.50446329e+02 -1.37112392e+02 -2.40833917e+02
 -1.69412836e+02  2.77350403e+02 -2.96348492e+02  1.13942224e+02
 -6.49869339e+00  4.77548944e+01  3.67570649e+02  3.96052185e+02
 -2.40656186e+02  1.78892845e+02  2.04060605e+01 -2.93813384e+02
  3.95173921e+02 -4.59395883e+01 -9.73269334e+01 -2.49232244e+02
 -1.05597941e+02 -2.22692502e+02 -9.71775497e+01 -2.97680536e+01
 -1.00060883e+02 -2.08714905e+02 -3.47356837e+02  1.03394542e+01
 -3.87401622e+02  8.41348912e+02 -2.81948002e+02 -6.43033034e+01
 -5.51208591e+02  1.37226446e+02  6.98926122e+02 -3.01966737e+02
 -2.29330009e+02 -2.80555591e+02  6.63801487e+02 -1.00178114e+02
 -3.14578458e+01 -8.92910667e+00  6.61665346e+02  6.54420140e+02
 -3.22172837e+01 -5.61092996e+0

## models with binary encoding

### RF

In [79]:
x_be = x_be.drop(['id_34_0', 'id_34_2', 'id_34_1'],axis = 1)
X_be_test = X_be_test.drop(['id_34', 'DeviceInfo_12'], axis = 1)

In [80]:
treeCount = 1000
m = 15

rf = RandomForestClassifier(max_features=m, min_samples_leaf=1, n_estimators=treeCount)
rf.fit(x_be, np.ravel(y_be))

Yhat_rf_be = rf.predict_proba(X_be_test)

submission_RF_be["isFraud"] = Yhat_rf_be[:, 1]

submission_RF_be.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionRFbe.csv", index = False)

### LR

In [82]:
lr = LogisticRegression(penalty='l2', max_iter=500, n_jobs=6, tol=1e-6, solver="sag")
lr.fit(x_be, np.ravel(y_be))

Yhat_lr_be = lr.predict_proba(X_be_test)
submission_be["isFraud"] = Yhat_lr_be[:, 1]

submission_be.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionLRbe.csv", index = False)


/Users/szeberinricsi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


NameError: name 'Yhat_lr' is not defined

In [84]:
submission_be["isFraud"] = Yhat_lr_be[:, 1]
submission_be.to_csv("/Users/szeberinricsi/Documents/UPS/S2/MOST_AA/ieee-fraud-detection/submissionLRbe.csv", index = False)
